# MCP를 통한 Internal Database 활용

이 노트북에서는 Strands Agents와 MCP (Model Context Protocol)를 사용하여 사내 데이터베이스 (PostgreSQL)와 연동하는 방법을 실습합니다.

## 학습 목표
- MCP를 통해 사내 데이터베이스를 Agent의 도구로 활용하는 방법 이해
- MCP 서버와 클라이언트를 통한 데이터베이스 연동 패턴 학습
- 관계형 데이터베이스의 쿼리를 몰라도 자연어로 쿼리를 실행하고 결과를 획득하는 과정 실습

## 실습 데이터셋 소개

의료 연구자들은 복잡한 데이터베이스 구조와 수많은 컬럼명을 모두 기억하기 어렵습니다. 이 실습에서는 다음과 같은 실제 의료 데이터를 사용하여, LLM의 도움으로 자연어만으로도 복잡한 데이터 분석이 가능함을 보여드립니다.

### chemotherapy_survival 테이블
화학요법 후 환자 생존 데이터 (50개 컬럼, 1,000+ 환자)

<img src="https://raw.githubusercontent.com/jikang-jeong/aws-q-mem-mcp-server/refs/heads/main/rdb_sample1.png" width="800" alt="chemotherapy_survival 데이터 예시">

### clinical_genomic 테이블
임상 및 유전체 통합 데이터 (200+ 컬럼, 500+ 환자)

<img src="https://raw.githubusercontent.com/jikang-jeong/aws-q-mem-mcp-server/refs/heads/main/rdb_sample2.png" width="800" alt="clinical_genomic 데이터 예시">

### 연구자의 도전과 LLM 에이전트의 접근방식

**🤔 연구자의 고민**: 
- "EGFR 돌연변이 환자의 생존율이 궁금한데, 어떤 컬럼을 조회해야 하지?"
- "50세 이상 흡연자 중 선암 환자는 몇 명일까? SQL을 어떻게 작성하지?"
- "LRIG1 유전자 발현량과 생존 기간의 상관관계를 보려면...?"


## 데이터베이스 스키마 상세 정보

### chemotherapy_survival 테이블
화학요법(항암치료) 후 환자들의 생존상태를 담고 있는 테이블

**컬럼 설명:**
- `lrig1`: Leucine-rich repeats and immunoglobulin-like domains 1 유전자 발현량
  - 세포 성장 조절 및 종양 억제 유전자로 작용할 수 있음
  - 항암 치료 반응성 또는 예후 지표로서 활용
  - LRIG1의 발현수준이 생존율과 상관관계를 가짐
- `survival_status`: 생존 상태 - true/false
- `survival_duration`: 생존 기간
- `expressiongroup`: 발현수준에 따라 나눈 그룹 - 0/1

### clinical_genomic 테이블
암 환자의 임상 및 유전체 정보를 포함하고 있는 테이블

**주요 컬럼들:**
- 다양한 유전자들의 발현량 컬럼
- `patient_affiliation`: 병원, 연구기관 등의 환자 소속정보
- `age_at_histological_diagnosis`: 조직학적 진단 당시 나이
- `weight_lbs`: 체중
- `gender`: 성별
- `ethnicity`: 인종
- `smoking_status`: 흡연 상태
- `pack_years`: 누적 흡연량
- `percent_gg`: 종양 내 GGO 비율
- `tumor_location_*`: 종양 위치
- `histology`: 조직학적 암 유형
- `pathological_*`: 병리적 병기/결과
- `histopathological_grade`: 암세포 분화 정도
- `lymphovascular_invasion`: 림프관 또는 혈관 침습 여부
- `pleural_invasion`: 흉막 침습 여부
- `egfr_mutation_status`: EGFR 유전자 돌연변이 상태
- `kras_mutation_status`: KRAS 유전자 돌연변이 상태
- `alk_translocation_status`: ALK 유전자 전좌 여부
- `adjuvant_treatment`: 보조 치료 여부
- `chemotherapy`: 항암화학요법 수행 여부
- `radiation`: 방사선 치료 여부
- `recurrence`: 암 재발 여부
- `recurrence_location`: 암 재발 부위
- `survival_status`: 생존 상태
- `time_to_death`: 진단 후 사망까지의 시간
- `days_between_ct_and_surgery`: CT 촬영일과 수술일 사이의 간격
- `survival_duration`: 전체 생존 기간

### 환경 설정

필요한 라이브러리와 의존성을 설치합니다.

In [ ]:
%pip install strands-agents strands-agents-tools psycopg2-binary --quiet

### MCP 서버 정의
데이터베이스 스키마 조회와 쿼리 실행을 위한 MCP 서버를 정의합니다.

MCP (Model Context Protocol)는 AI 에이전트가 외부 도구와 통신할 수 있게 해주는 프로토콜입니다.


In [ ]:

# AWS SDK
import boto3
from botocore.config import Config

# Strands Agents
from strands import Agent
from strands.models import BedrockModel
from strands.tools.mcp import MCPClient

# MCP
from mcp import stdio_client, StdioServerParameters


다음 rds endpoint를 워크샵 메인 페이지 Outputs AuroraClusterEndpoint 값을 확인해 DB_CONFIG 변수에 할당 합니다.
<img src="https://raw.githubusercontent.com/jikang-jeong/aws-q-mem-mcp-server/refs/heads/main/rdb_sample0.png" width="800" alt="rds endpoint">



In [ ]:
%%writefile mcp_server_internal_db.py

import logging
import sys
from mcp.server.fastmcp import FastMCP
import psycopg2
from collections import defaultdict

# Database configuration
DB_CONFIG = {
    "host": "YOUR_RDS_ENDPOINT",  # ⚠️ 중요: RDS 엔드포인트 설정 Outputs Key: AuroraClusterEndpoint 값을 입력하세요.
    "port": 5432,
    "database": "agentdb",
    "user": "dbadmin",
    "password": "postgres"
}

logging.basicConfig(
    level=logging.INFO,
    format='%(filename)s:%(lineno)d | %(message)s',
    handlers=[logging.StreamHandler(sys.stderr)]
)
logger = logging.getLogger("internal_db_mcp")

try:
    mcp = FastMCP(name="internal_db_tool")
    logger.info("internal database Postgres MCP server initialized successfully")
except Exception as e:
    logger.error(f"Error: {str(e)}")

@mcp.tool
async def fetch_table_schema() -> dict:
    """Postgres public 스키마의 테이블과 컬럼 정보를 조회하는 도구입니다.
    - 쿼리를 자동 생성할 때 테이블 구조가 필요하면 이 도구를 사용하세요.
    - 단순히 데이터 조회만 하는 경우에는 호출할 필요가 없습니다.
    """
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        with conn.cursor() as cursor:
            cursor.execute("""
                           SELECT
                               c.relname AS table_name,
                               a.attname AS column_name,
                               pg_catalog.format_type(a.atttypid, a.atttypmod) AS data_type,
                               d.description AS column_comment
                           FROM pg_catalog.pg_attribute a
                                    JOIN pg_catalog.pg_class c ON a.attrelid = c.oid
                                    JOIN pg_catalog.pg_namespace n ON c.relnamespace = n.oid
                                    LEFT JOIN pg_catalog.pg_description d ON a.attrelid = d.objoid AND a.attnum = d.objsubid
                           WHERE a.attnum > 0 AND NOT a.attisdropped
                             AND n.nspname = 'public'
                             AND c.relkind = 'r'
                           ORDER BY c.relname, a.attnum;
                           """)

            rows = cursor.fetchall()
            # 테이블별로 그룹화
            tables = defaultdict(list)
            for table_name, column_name, data_type, column_comment in rows:
                tables[table_name].append({
                    "column_name": column_name,
                    "data_type": data_type,
                    "comment": column_comment
                })

            result = [{"table_name": tbl, "columns": cols} for tbl, cols in tables.items()]
            return {"message": result, "status": "success"}

    except Exception as e:
        return {"error": str(e)}
    finally:
        if 'conn' in locals():
            conn.close()

@mcp.tool
async def execute_postgres_query(query: str) -> dict:
    """사용자가 요청한 SQL 쿼리를 실행하는 도구입니다.
    - 쿼리가 명확히 주어진 경우 이 도구만 호출하면 됩니다.
    - 쿼리를 자동 생성해야 하는 경우, 필요하다면 먼저 fetch_table_schema를 호출해 스키마를 확인한 뒤 실행하세요."""
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cur = conn.cursor()
        cur.execute(query)
        result = cur.fetchall()
        conn.commit()
        return {
            "message": "\n".join(str(row) for row in result),
            "status": "success"
        }
    except Exception as e:
        return {
            "message": str(e)
        }
    finally:
        if 'cur' in locals():
            cur.close()
        if 'conn' in locals():
            conn.close()


if __name__ == "__main__":
    mcp.run()

### MCP 클라이언트 설정

사내 데이터베이스에 대한 MCP 클라이언트를 설정합니다.
클라이언트는 위에서 정의한 MCP 서버와 통신하여 데이터베이스 작업을 수행합니다.

In [ ]:
# MCP 서버 파라미터 설정
internalDb_server_params = StdioServerParameters(
    command="python",
    args=["mcp_server_internal_db.py"]
)

In [ ]:
# MCP 클라이언트 생성
internalDb_client = MCPClient(internalDb_server_params)

In [ ]:
# Bedrock 모델 설정
model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    region_name="us-east-1"
)

### 데이터베이스 에이전트 생성
PostgreSQL 데이터베이스와 연동하는 Strands 에이전트를 생성합니다.
이 에이전트는 MCP 클라이언트를 도구로 사용하여 자연어 질문을 SQL 쿼리로 변환하고 실행합니다.

In [ ]:
# 사내 데이터베이스 탐색 전문 에이전트
internalDb_agent = Agent(
    name="internalDbAgent",
    system_prompt="""당신은 의료 데이터베이스 전문가 AI입니다.

**중요**: 모든 데이터베이스 관련 질문에 대해 반드시 다음 중 하나의 도구를 사용해야 합니다:
1. fetch_table_schema() - 스키마 정보가 필요할 때
2. execute_postgres_query() - 실제 데이터 조회가 필요할 때

PostgreSQL 데이터베이스에 저장된 임상 및 유전체 데이터를 분석하고 쿼리할 수 있습니다.

주요 기능:
1. 데이터베이스 스키마 탐색 및 이해
2. 자연어 질문을 SQL 쿼리로 변환
3. 임상 데이터 분석 및 통계 제공
4. 유전체 데이터 패턴 분석

데이터베이스 정보:
- chemotherapy_survival: 화학요법 후 환자 생존 데이터
  - LRIG1: 유전자 발현량
  - Survival_Status: 생존 상태 (true/false)
  - Survival_Duration: 생존 기간
  - ExpressionGroup: 발현 그룹 (0/1)

- clinical_genomic: 임상 및 유전체 정보
  - 다양한 유전자 발현량 컬럼들
  - 환자 임상 정보 (나이, 성별, 흡연상태 등)
  - 종양 정보 (위치, 병리학적 병기 등)
  - 치료 정보 (화학요법, 방사선치료 등)

항상 도움이 되고 정확한 정보를 제공하며, 의료 데이터의 중요성을 인식하고 신중하게 분석하세요.""",
    model=model,
    tools=[internalDb_client]
)


## 데이터베이스 에이전트 테스트
데이터베이스 에이전트를 다양한 쿼리로 테스트해봅니다.

**참고**: `YOUR_RDS_ENDPOINT`를 실제 RDS 엔드포인트로 변경한 후 테스트를 진행하세요.

### Test 1: 데이터베이스 연결 및 스키마 조회

In [ ]:
# internal db (postgres) 에이전트 테스트
print("=== 사내 데이터베이스 에이전트 * Database Connection and Schema * 테스트 ===")
db_result = internalDb_agent("데이터베이스에 어떤 테이블들이 있는지 알려주세요")
print("\n" + "="*50 + "\n")

### Test 2: 간단한 데이터베이스 쿼리

In [ ]:
# internal db (postgres) 에이전트 테스트
print("=== 사내 데이터베이스 에이전트 * Simple Database Query * 테스트 ===")
db_result = internalDb_agent("현재 데이터베이스 시간을 알려주세요")
print("\n" + "="*50 + "\n")

### Test 3: 임상 데이터 분석

In [ ]:
# internal db (postgres) 에이전트 테스트
print("=== 사내 데이터베이스 에이전트 * Clinical Data Analysis * 테스트 ===")
db_result = internalDb_agent("선암(Adenocarcinoma) 진단을 받은 환자의 평균 연령은 얼마입니까?")
print("\n" + "="*50 + "\n")

### Test 4: 복합 분석 쿼리

In [ ]:

# internal db (postgres) 에이전트 테스트
print("=== 사내 데이터베이스 에이전트 * Complex Analysis Query * 테스트 ===")
db_result = internalDb_agent("50세 이상 진단을 받은 환자의 수는 몇 명이며, 흡연 여부는 어떻게 됩니까?")
print("\n" + "="*50 + "\n")

### Test 5: 생존 분석

In [ ]:

# internal db (postgres) 에이전트 테스트
print("=== 사내 데이터베이스 에이전트 * Survival Analysis * 테스트 ===")
db_result = internalDb_agent("화학요법 후 생존한 환자와 사망한 환자의 LRIG1 유전자 발현량 평균을 비교해주세요")
print("\n" + "="*50 + "\n")

### Test 6: 돌연변이 분석

In [ ]:

# internal db (postgres) 에이전트 테스트
print("=== 사내 데이터베이스 에이전트 * Test mutation analysis * 테스트 ===")
db_result = internalDb_agent("EGFR 돌연변이를 가진 환자들의 비율과 이들의 생존율을 분석해주세요")
print("\n" + "="*50 + "\n")


## 결론

이 노트북에서는 Strands Agents 프레임워크를 사용하여 PostgreSQL 데이터베이스와 연동하는 에이전트를 성공적으로 구현했습니다.

### 주요 구현 내용:

#### 1. 데이터베이스 도구 구현
- **fetch_table_schema**: 데이터베이스 스키마 탐색 도구
- **execute_postgres_query**: SQL 쿼리 실행 도구

#### 2. 에이전트 기능
- 자연어 질문을 SQL 쿼리로 변환
- 임상 및 유전체 데이터 분석
- 생존 분석 및 돌연변이 분석
- 통계적 분석 및 패턴 발견

#### 3. 데이터셋 특징
- **chemotherapy_survival**: 화학요법 후 환자 생존 데이터
- **clinical_genomic**: 초기 단계 비소세포폐암 환자 코호트 데이터
- 종양 조직의 유전자 발현 데이터
- 환자의 임상 및 인구통계학적 정보

### Strands 프레임워크의 장점:

1. **간편한 도구 정의**: `@tool` 데코레이터를 사용한 직관적인 도구 구현
2. **직접적인 통합**: Lambda 함수 없이 직접 데이터베이스 연결
3. **유연한 에이전트 구성**: 시스템 프롬프트와 도구의 조합으로 전문화된 에이전트 생성
4. **쉬운 테스트**: 노트북 환경에서 직접 테스트 가능
5. **비용 효율성**: Lambda 함수 비용 제거
6. **빠른 실행**: Lambda 콜드 스타트 지연 제거

### 사용 예시:
- 선암종 환자들의 평균 연령 조회
- 편평세포암종 환자들의 가장 흔한 병리학적 T 병기 분석
- EGFR 돌연변이를 가진 환자들의 비율 계산
- 화학요법 후 생존율과 유전자 발현량 상관관계 분석

이 구현을 통해 의료 연구자들은 복잡한 SQL을 작성하지 않고도 자연어로 데이터베이스를 쿼리하고 분석할 수 있게 되었습니다.